## VI. TRAINING A SIMPLE INHARMONICITY-BASED CLASSIFIER <br />

<div style="text-align:justified;text-indent:10px"> On a guitar tuned to standard tuning, one can play notes from E2 (82.4 Hz, 6th string, open) to E5 (659.3 Hz, 1st string, fretted at the 12th position), if we restrict ourselves to the first 12 frets. From the geometry of the guitar, each note in the above range can only be played at a finite number of positions, typically up to 3. Thus, for each fundamental frequency $f_0$ in the range [82.4, 659.3] we have a list of pairs $\{(n_i, p_i)\}_{i \in I}$ that represents all possible fingerings for this particular $f_0$. The elements of these pairs are the string number $n_i$ (between 1 and 6) and the fretboard position $p_i$ (between 0 and 12). Therefore, the fingering estimation reduces to a classification problem. </div><br />

<div style="text-align:justified;"> In the previous chapter, we developed a method to track the partials of a monophonic guitar sound in successive time frames and to estimate the corresponding inharmonicity coefficient $\hat{B}$. Assembling all the tools we have seen, we now construct a simple inharmonicity-based classifier. The resulting system may be seen as a one-feature classifier: it decided which fingering was used only using the estimated inharmonicity. In order to do this, let us first define a measure that only depends on the string rather than the particular position being used. We consider a new constant $\gamma$ given by </div><br />

\begin{equation}
    \gamma = BL^2 = \frac{\pi^2 E S K^2}{T}.
\end{equation}

<br/><div style="text-align:justified;"> $\gamma$ only depends on string properties (elastic modulus $E$, cross-sectional area $S$, radius of gyration $K$ and tension $T$). The idea is now to learn the 6 constants $\gamma_1$ to $\gamma_6$ from data. </div><br />

<div style="text-align:justified;"> Our dataset consists of 780 recordings (10 per position, up to the 12th fret on each of the 6 strings) of isolated notes, in the range E2-E5. We used a Washburn D10 Dreadnought acoustic guitar directly plugged into an audio interface M-audio M-track 8 to perform all the recordings. We divide these recordings into training recordings (forming the training set) and test recordings (forming the test set). We chose a 50-50 train-test split, i.e 50% of the recordings are used for training and 50% for testing. We now describe the training and testing procedures in more details. </div>


##### Training procedure:

- Given all the training recordings of a fingering $(n_i, p_i)$, we use our inharmonicity estimation method to estimate the inharmonicity constant $\hat{B}_{(n_i, p_i)}$ of that fingering. We have to choose which method we want to use (either the average or the polynomial fit, with either Goertzel or MUSIC).


- Given all the estimated inharmonicity constants $\hat{B}_{(n_i, p_i)}$ for one string, we can then compute the corresponding constant gamma as an average, 


\begin{equation}
    \hat{\gamma}_i = \frac{1}{13} \sum_{m = 0}^{12} \hat{B}_{(n_i, p_m)} L^2_m.
\end{equation}


- Once this learning phase is done, we have at our disposal 6 constants $\gamma_1$ to $\gamma_6$, representing the "inharmonicity profiles" of each string.


##### Testing procedure:


- Given a test recording, we first process it in the same fashion as the training recordings: we estimate the fundamental, split it into frames, track the partials in all the frames, and finally estimate the inharmonicity $\hat{B}_{test}$. 


- For all potential fingering candidates $\{(n_i, p_i)\}_{i \in I}$ of the estimated $f_0$, we compare the estimated inharmonicity coefficient $\hat{B}_{test}$ to the $\{\gamma_i / L_i^2\}_{i \in I}$, where $\gamma_i$ is the corresponding string inharmonicity profile and $L_i$ the string length corresponding to the fretboard position $p_i$. The best candidate, i.e the one with the closest inharmonicity coefficient, is declared to be the fingering that was used.


- This testing procedure is repeated for all the test recordings. Our performance measure will be the number of misclassifications. In our context, a misclassification is defined as a fingering estimated incorrectly.

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt

from scipy.io import wavfile

from pitch_estimation import *
from partials_tracking import *
from misc import *

import pickle

### VI.I Global parameters

In [2]:
# general parameters
string_full_length = .64
max_mode = 8
modes_num = np.arange(1, max_mode + 1)

# dataset parameters & info
PATH_RECORDINGS = '../recordings/'
num_rec_per_pos = 10

# algorithms parameters
NFFT = 2**13
tol = .05
max_num_peaks = 10

M = 1000
freq_res = .05

delta_f = 5

frame_duration = .75

partial_tracking_method = 'goertzel'
inharm_est_method = 'average'

# compute the string length for all positions on the
# guitar neck (up to the 12th fret)
all_string_lengths = np.zeros(13)
for n in range(13):
    all_string_lengths[n] = compute_string_length(string_full_length, n)

### VI.II Train-test split <br />

<div style="text-align:justified;"> As we said, we first need to divide our dataset into training and test recordings. We choose a 50-50 split, i.e 50% of the recordings are used for training, and 50% for testing. To test equally all fingerings, we select randomly 5 out of 10 recordings of each fingering for training, and the remaining 5 for testing. </div>

In [6]:
# each entry is an array with the recording indices/numbers
# used for training and testing, respectively
train_indices = np.zeros([6, 13, 5])
test_indices = np.zeros([6, 13, 5])

# for all possible fingering we consider
for n in range(6):
    for m in range(13):
        
        # randomly permute the recording indices
        permuted_indices = np.random.permutation(np.arange(num_rec_per_pos))
        
        # pick the first 5 for training and the remaining 5 for testing
        train_indices[n, m] = permuted_indices[:5]
        test_indices[n, m] = permuted_indices[5:]

### VI.III Training our classifier <br />

<div style="text-align:justified;"> We now turn to the training of our model. Using our partial tracking and inharmonicity estimation methods, we estimate the constant $\gamma_i$ of each string. Once this learning phase is done, we will have at our disposal 6 constants, $\gamma_1$ to $\gamma_6$, representing the "inharmonicity profiles" of each string. </div>

In [11]:
# initializations
gammas = np.zeros(6)
b_hats = np.zeros([6, 13])

print("\nTraining our model...")

# We iterate over all possible fingerings (over the 6 strings and the first 13 positions)
for n in range(6):
    
    print("\nString n°{str_num}".format(str_num=n+1))
    
    for m in range(13):
        
        print("\n\tFret n°{fret_num}".format(fret_num=m))
        
        # initializations for the current fingering
        file_folder = 'string_' + str(n + 1) + '/string_' + str(n + 1) + '_fret_' + str(m) + '/'
        file_name_prefix = 'string_' + str(n + 1) + '_fret_' + str(m) + '_take_'
        
        curr_train_idx = train_indices[n, m]
        curr_string_length = compute_string_length(string_full_length, m)
        
        stacked_freq_ratios = []
        stacked_b_hats = []
        
        
        # We use all the training recordings of that fingering to train the model
        for rec_num in curr_train_idx:
            
            # read the recording from the data folder
            file_name = file_name_prefix + str(int(rec_num + 1)) + '.wav'            
            fs_recording, current_recording = wavfile.read(PATH_RECORDINGS + file_folder + file_name)
            y = current_recording / max(current_recording)
            
            # extract the first 1.5 seconds of the recording
            y = y[:int(1.5*fs_recording)]
            
            # initializations
            tot_num_samp = y.shape[0]
            num_samp_per_frame = int(fs_recording * frame_duration)
            window = np.hamming(num_samp_per_frame)
            hop_size = int(num_samp_per_frame/2)
            num_frames = int((tot_num_samp - num_samp_per_frame)/hop_size)
            
            # estimate the fundamental frequency
            estimated_fundamental = estimate_fundamental(np.abs(np.fft.fft(np.hamming(y.shape[0]) * y, NFFT)), fs_recording, \
                                                 NFFT, tol, max_num_peaks)
            
            #TMP: PROBLEM WITH THE FIRST STRING (OPEN POSITION ONLY)
            if(n == 0 and m == 0):
                estimated_fundamental = 329.6
            
            print("\n\t\tEstimated fundamental [Hz]: {est_fund}".format(est_fund=estimated_fundamental))
            
            # track the partials in successive time frames
            estimated_partials, partials_amplitudes = partials_tracking(y, fs_recording, frame_duration, \
                                                                        estimated_fundamental, max_mode, delta_f, \
                                                                        partial_tracking_method, M, freq_res)
            
            # depending on the chosen inharmonicity estimation method, we store different things
            if(inharm_est_method == 'average'):
                
                # stack up all the estimates of the inharmonicity constant
                curr_stacked_b_hats = np.hstack([((estimated_partials[1:, n]/estimated_partials[0, n])**2 \
                   - modes_num[1:]**2)/modes_num[1:]**4 for n in range(num_frames)])
                
                # store
                stacked_b_hats.append(curr_stacked_b_hats)
                
            elif(inharm_est_method == 'poly'):
                
                # stack up the squared frequency ratios (fk/f1)^2
                curr_freq_ratios = np.hstack([(estimated_partials[:, n]/estimated_partials[0, n])**2 for n in range(num_frames)])
                
                # store
                stacked_freq_ratios.append(curr_freq_ratios)
                
            else:
                print("Error: undefined inharmonicity estimation method.")
                break
            
        
        # estimate the inharmonicity coefficient
        if(inharm_est_method == 'average'):
            stacked_b_hats = np.hstack(stacked_b_hats)
            positive_b_hats = stacked_b_hats[np.where(stacked_b_hats > 0)]
            b_hat = np.mean(positive_b_hats)
        elif(inharm_est_method == 'poly'):
            b_hat = estimate_inharmonicity_poly(5, stacked_freq_ratios, num_frames, max_mode)
        else:
            print("Error: undefined inharmonicity estimation method")
            break
        
        print("\n\tEstimated inharmonicity coefficient: {b}".format(b=b_hat))
        
        # store the estimated inharmonicity coefficient
        b_hats[n, m] = b_hat
        
    
    # estimate the constant gamma of the string by averaging the gammas of all positions
    gammas[n] = np.nanmean(b_hats[n, :] * all_string_lengths**2)


Training our model...

String n°1

	Fret n°0

		Estimated fundamental [Hz]: 329.6

		Estimated fundamental [Hz]: 329.6

		Estimated fundamental [Hz]: 329.6

		Estimated fundamental [Hz]: 329.6

		Estimated fundamental [Hz]: 329.6

	Estimated inharmonicity coefficient: 4.816796898458903e-06

	Fret n°1

		Estimated fundamental [Hz]: 349.2

		Estimated fundamental [Hz]: 349.2

		Estimated fundamental [Hz]: 349.2

		Estimated fundamental [Hz]: 349.2

		Estimated fundamental [Hz]: 349.2

	Estimated inharmonicity coefficient: 5.530780383618304e-05

	Fret n°2

		Estimated fundamental [Hz]: 370.0

		Estimated fundamental [Hz]: 370.0

		Estimated fundamental [Hz]: 370.0

		Estimated fundamental [Hz]: 370.0

		Estimated fundamental [Hz]: 370.0

	Estimated inharmonicity coefficient: 0.00011505447347931835

	Fret n°3

		Estimated fundamental [Hz]: 392.0

		Estimated fundamental [Hz]: 392.0

		Estimated fundamental [Hz]: 392.0

		Estimated fundamental [Hz]: 392.0

		Estimated fundamental [Hz]: 392

C:\Users\rafilatty\Anaconda3\lib\site-packages\numpy\core\_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
C:\Users\rafilatty\Anaconda3\lib\site-packages\numpy\core\_methods.py:70: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)



	Estimated inharmonicity coefficient: nan

	Fret n°2

		Estimated fundamental [Hz]: 220.0

		Estimated fundamental [Hz]: 220.0

		Estimated fundamental [Hz]: 220.0

		Estimated fundamental [Hz]: 220.0

		Estimated fundamental [Hz]: 220.0

	Estimated inharmonicity coefficient: 2.5453703270809926e-05

	Fret n°3

		Estimated fundamental [Hz]: 233.1

		Estimated fundamental [Hz]: 233.1

		Estimated fundamental [Hz]: 233.1

		Estimated fundamental [Hz]: 233.1

		Estimated fundamental [Hz]: 233.1

	Estimated inharmonicity coefficient: 1.1041698090633551e-05

	Fret n°4

		Estimated fundamental [Hz]: 246.9

		Estimated fundamental [Hz]: 246.9

		Estimated fundamental [Hz]: 246.9

		Estimated fundamental [Hz]: 246.9

		Estimated fundamental [Hz]: 246.9

	Estimated inharmonicity coefficient: 0.00019820092268744074

	Fret n°5

		Estimated fundamental [Hz]: 261.6

		Estimated fundamental [Hz]: 261.6

		Estimated fundamental [Hz]: 261.6

		Estimated fundamental [Hz]: 261.6

		Estimated fundamental

In [14]:
print("\nTraining phase completed!")
print("\nEstimated gammas: {gammas}".format(gammas=gammas))


Training phase completed!

Estimated gammas: [  1.55015727e-05   2.00590872e-05   1.92337062e-05   1.10364570e-04
   9.44898173e-05   8.85864412e-05]


In [3]:
# OPTIONAL: store the estimated gammas and the train-test indices
#pickle.dump(gammas, open('data_dumps/gammas', 'wb'))
#pickle.dump(train_indices, open('data_dumps/train_indices', 'wb'))
#pickle.dump(test_indices, open('data_dumps/test_indices', 'wb'))

### VI.IV Testing the model <br />

<div style="text-align:justified;"> We can now test our model on our test set. As we said, we first process the test recordings in the same fashion as we have done for training recordings, i.e first estimate the fundamental, extract the partials frame by frame, then estimate the inharmonicity constant $\hat{B}$. Then, for the given estimated fundamental, we look at all the potential candidates (all possible fingerings of that particular note) and take the closest in the sense of inharmonicity. </div>

In [16]:
# OPTIONAL: load the estimated gammas (output of the training phase) 
# and the train-test indices to avoid retraining the model

#gammas = pickle.load(open('data_dumps/gammas', 'rb'))
#train_indices = pickle.load(open('data_dumps/train_indices', 'rb'))
#test_indices = pickle.load(open('data_dumps/test_indices', 'rb'))

##### Testing a single recording <br />

<div style="text-align:justified;"> In this subsection, we test our model on a single, hand-picked recording in order to visualize what happens. </div>


Estimated fundamental [Hz]: 110.0

	Estimated inharmonicity constant: 0.00016195719519301202

	Potential fingerings candidates: [(6, 5), (5, 0)]
	Corresponding inharmonicity coefficients: [ 0.00038536  0.00023069]


Error! The true fingering was (6, 5) but we decided (5, 0)


In [21]:
# test recording parameters
str_num = 6
fret_num = 5
take_num = int(test_indices[str_num - 1, fret_num][1]) + 1

file_folder = 'string_' + str(str_num) + '/string_' + str(str_num) + '_fret_' + str(fret_num) + '/'
file_name = 'string_' + str(str_num) + '_fret_' + str(fret_num) + '_take_' + str(take_num) + '.wav'

curr_string_length = compute_string_length(string_full_length, fret_num)

# read the recording
fs_recording, current_recording = wavfile.read(PATH_RECORDINGS + file_folder + file_name)
y = current_recording / max(current_recording)

# extract the first 1.5 seconds of the recording
y = y[:int(1.5*fs_recording)]

# initializations and parameters
stacked_freq_ratios = []
tot_num_samp = y.shape[0]
num_samp_per_frame = int(fs_recording * frame_duration)
window = np.hamming(num_samp_per_frame)
hop_size = int(num_samp_per_frame / 2)
num_frames = int((tot_num_samp - num_samp_per_frame) / hop_size)

# estimate the fundamental frequency
estimated_fundamental = estimate_fundamental(np.abs(np.fft.rfft(y * np.hamming(tot_num_samp), NFFT)), fs_recording, NFFT, tol, max_num_peaks)
print("\nEstimated fundamental [Hz]: {est_fund}".format(est_fund=estimated_fundamental))

# track the partials in successive time frames
estimated_partials, partials_amplitudes = partials_tracking(y, fs_recording, frame_duration, estimated_fundamental, max_mode, \
                                                   delta_f, partial_tracking_method, M, freq_res)

# estimate the inharmonicity coefficient
if(inharm_est_method == 'average'):
    
    # stack
    stacked_b_hats = np.hstack([((estimated_partials[1:, n]/estimated_partials[0, n])**2 \
           - modes_num[1:]**2)/modes_num[1:]**4 for n in range(num_frames)])

    # remove the negative ones (declared as "outliers")
    positive_b_hats = stacked_b_hats[np.where(stacked_b_hats > 0)]

    # compute the average
    b_hat_ = np.mean(positive_b_hats)
    
elif(inharm_est_method == 'poly'):
    
    # stack the squared frequency ratios
    freq_ratios = [(estimated_partials[:, n]/estimated_partials[0, n])**2 for n in range(num_frames)]
    stacked_freq_ratios.append(np.hstack(freq_ratios))
    
    # polynomial fit
    b_hat = estimate_inharmonicity_poly(1, stacked_freq_ratios, num_frames, max_mode)
    
print("\n\tEstimated inharmonicity constant: {b}".format(b=b_hat))

# find the best fingering using our trained model
b_candidates, best_fingering = find_best_fingering(gammas, estimated_fundamental, b_hat, string_full_length)

num_candidates = len(b_candidates)

# visualization
plt.figure(figsize=(10, 6))

for n in range(num_candidates):
    norm_freq_diff_candidate = (modes_num * estimated_fundamental * np.sqrt(1 + b_candidates[n] * modes_num**2) - \
        modes_num * estimated_fundamental) / estimated_fundamental
    plt.plot(modes_num, norm_freq_diff_candidate, 'bx--')

norm_freq_diff_recording = (modes_num * estimated_fundamental * np.sqrt(1 + b_hat * modes_num**2) - \
        modes_num * estimated_fundamental) / estimated_fundamental 
plt.plot(modes_num, norm_freq_diff_recording, 'ro--')
plt.xlabel('Partial number')
plt.ylabel('Normalized frequency difference')
plt.title('INHARMONICITY CURVES FOR POTENTIAL FINGERINGS')
plt.tight_layout()
plt.show()

# => Test if we guessed correctly !
if(best_fingering != (str_num, fret_num)):
    print("\n\tError! The true fingering was {true_fing} but we decided {est_fing}".format(true_fing=(str_num, fret_num), \
                                                                                                   est_fing=best_fingering))
else:
    print("Fingering estimated correctly!")

##### Testing the entire test set <br />

<div style="text-align;justified;"> In order to measure the performance of our model, we now test it on all our test recordings. </div>

In [23]:
# initializations
num_errors_per_string = np.zeros(6)

print("\nTesting our model...")

# We test all possible fingerings (over the 6 strings and the first 13 positions)
for n in range(6):
    
    print("\nString n°{str_num}".format(str_num=n+1))
    
    for m in range(13):
        
        print("\n\tFret n°{fret_num}".format(fret_num=m))
        
        # initializations for the current fingering
        file_folder = 'string_' + str(n + 1) + '/string_' + str(n + 1) + '_fret_' + str(m) + '/'
        file_name_prefix = 'string_' + str(n + 1) + '_fret_' + str(m) + '_take_'
        curr_test_idx = test_indices[n, m]
        curr_string_length = compute_string_length(string_full_length, m)
        
        # We evaluate one test recording at a time
        for rec_num in curr_test_idx:
            
            # initializations
            stacked_freq_ratios = []
            stacked_b_hats = []
            
            # read the recording from the data folder
            file_name = file_name_prefix + str(int(rec_num + 1)) + '.wav'            
            fs_recording, current_recording = wavfile.read(PATH_RECORDINGS + file_folder + file_name)
            y = current_recording / max(current_recording)
            
            # extract the first 1.5 seconds of the recording
            y = y[:int(1.5*fs_recording)]
            
            # parameters
            tot_num_samp = y.shape[0]
            num_samp_per_frame = int(fs_recording * frame_duration)
            window = np.hamming(num_samp_per_frame)
            hop_size = int(num_samp_per_frame/2)
            num_frames = int((tot_num_samp - num_samp_per_frame)/hop_size)
            
            # estimate the fundamental frequency
            estimated_fundamental = estimate_fundamental(np.abs(np.fft.rfft(y * np.hamming(tot_num_samp), NFFT)), fs_recording, \
                                                 NFFT, tol, max_num_peaks)
            
            # TMP: PROBLEM WITH THE FIRST STRING
            if(n == 0 and m == 0):
                estimated_fundamental = 329.6
            
            print("\n\t\tEstimated fundamental [Hz]: {est_fund}".format(est_fund=estimated_fundamental))
            
            # track the partials in successive time frames
            estimated_partials, partials_amplitudes = partials_tracking(y, fs_recording, frame_duration, estimated_fundamental, max_mode, \
                                                   delta_f, partial_tracking_method, M, freq_res)
            
            # estimate the inharmonicity coefficient
            if(inharm_est_method == 'average'):
                
                # stack up all the estimates of the inharmonicity constant
                stacked_b_hats = np.hstack([((estimated_partials[1:, n]/estimated_partials[0, n])**2 \
                   - modes_num[1:]**2)/modes_num[1:]**4 for n in range(num_frames)])
                
                # remove the negative estimates
                positive_b_hats = stacked_b_hats[np.where(stacked_b_hats > 0)]
                
                # average
                b_hat = np.nanmean(positive_b_hats)
                
            elif(inharm_est_method == 'poly'):
                
                # stack up the squared frequency ratios (fk/f1)^2
                stacked_freq_ratios = np.hstack([(estimated_partials[:, n]/estimated_partials[0, n])**2 for n in range(num_frames)])
                
                b_hat = estimate_inharmonicity_poly(1, stacked_freq_ratios, num_frames, max_mode)
                
            else:
                print("Error: undefined inharmonicity estimation method.")
                break
                
            print("\t\tEstimated inharmonicity constant: {b}".format(b=b_hat))
            
            # find the best fingering using our trained model
            b_candidates, best_fingering = find_best_fingering(gammas, estimated_fundamental, b_hat, string_full_length)
            
            # => Test if we guessed correctly !
            if(best_fingering != (n + 1, m)):
                print("\n\t\tError! The true fingering was {true_fing} but we decided {est_fing}".format(true_fing=(n + 1, m), \
                                                                                               est_fing=best_fingering))
                num_errors_per_string[n] += 1
            else:
                # fingering correctly estimated
                print("\n\t\tFingering correctly estimated!")


Testing our model...

String n°1

	Fret n°0

		Estimated fundamental [Hz]: 329.6


C:\Users\rafilatty\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:675: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)


		Estimated inharmonicity constant: nan

	Potential fingerings candidates: [(3, 9), (2, 5), (1, 0)]
	Corresponding inharmonicity coefficients: [  1.32816501e-04   8.72593104e-05   3.78456365e-05]

		Error! The true fingering was (1, 0) but we decided (3, 9)

		Estimated fundamental [Hz]: 329.6


C:\Users\rafilatty\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:675: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)


		Estimated inharmonicity constant: nan

	Potential fingerings candidates: [(3, 9), (2, 5), (1, 0)]
	Corresponding inharmonicity coefficients: [  1.32816501e-04   8.72593104e-05   3.78456365e-05]

		Error! The true fingering was (1, 0) but we decided (3, 9)

		Estimated fundamental [Hz]: 329.6
		Estimated inharmonicity constant: 1.9634471722985225e-06

	Potential fingerings candidates: [(3, 9), (2, 5), (1, 0)]
	Corresponding inharmonicity coefficients: [  1.32816501e-04   8.72593104e-05   3.78456365e-05]

		Fingering correctly estimated!

		Estimated fundamental [Hz]: 329.6
		Estimated inharmonicity constant: 6.525695435487255e-06

	Potential fingerings candidates: [(3, 9), (2, 5), (1, 0)]
	Corresponding inharmonicity coefficients: [  1.32816501e-04   8.72593104e-05   3.78456365e-05]

		Fingering correctly estimated!

		Estimated fundamental [Hz]: 329.6
		Estimated inharmonicity constant: 6.76716623787988e-06

	Potential fingerings candidates: [(3, 9), (2, 5), (1, 0)]
	Corresponding in

C:\Users\rafilatty\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:675: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)


		Estimated inharmonicity constant: nan

	Potential fingerings candidates: [(4, 9), (3, 4), (2, 0)]
	Corresponding inharmonicity coefficients: [  7.62111880e-04   7.45403583e-05   4.89723809e-05]

		Error! The true fingering was (2, 0) but we decided (4, 9)

		Estimated fundamental [Hz]: 246.9
		Estimated inharmonicity constant: 1.6961220631432627e-06

	Potential fingerings candidates: [(4, 9), (3, 4), (2, 0)]
	Corresponding inharmonicity coefficients: [  7.62111880e-04   7.45403583e-05   4.89723809e-05]

		Fingering correctly estimated!

		Estimated fundamental [Hz]: 246.9
		Estimated inharmonicity constant: 9.071001277439344e-06

	Potential fingerings candidates: [(4, 9), (3, 4), (2, 0)]
	Corresponding inharmonicity coefficients: [  7.62111880e-04   7.45403583e-05   4.89723809e-05]

		Fingering correctly estimated!

	Fret n°1

		Estimated fundamental [Hz]: 261.6
		Estimated inharmonicity constant: 1.9837075936925052e-05

	Potential fingerings candidates: [(4, 10), (3, 5), (2, 1)]
	Co

C:\Users\rafilatty\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:675: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)


		Estimated inharmonicity constant: nan

	Potential fingerings candidates: [(5, 11), (4, 6), (3, 1)]
	Corresponding inharmonicity coefficients: [  8.22087963e-04   5.38892819e-04   5.27078306e-05]

		Error! The true fingering was (3, 1) but we decided (5, 11)

		Estimated fundamental [Hz]: 207.6


C:\Users\rafilatty\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:675: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)


		Estimated inharmonicity constant: nan

	Potential fingerings candidates: [(5, 11), (4, 6), (3, 1)]
	Corresponding inharmonicity coefficients: [  8.22087963e-04   5.38892819e-04   5.27078306e-05]

		Error! The true fingering was (3, 1) but we decided (5, 11)

		Estimated fundamental [Hz]: 207.6


C:\Users\rafilatty\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:675: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)


		Estimated inharmonicity constant: nan

	Potential fingerings candidates: [(5, 11), (4, 6), (3, 1)]
	Corresponding inharmonicity coefficients: [  8.22087963e-04   5.38892819e-04   5.27078306e-05]

		Error! The true fingering was (3, 1) but we decided (5, 11)

		Estimated fundamental [Hz]: 207.6


C:\Users\rafilatty\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:675: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)


		Estimated inharmonicity constant: nan

	Potential fingerings candidates: [(5, 11), (4, 6), (3, 1)]
	Corresponding inharmonicity coefficients: [  8.22087963e-04   5.38892819e-04   5.27078306e-05]

		Error! The true fingering was (3, 1) but we decided (5, 11)

		Estimated fundamental [Hz]: 207.6


C:\Users\rafilatty\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:675: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)


		Estimated inharmonicity constant: nan

	Potential fingerings candidates: [(5, 11), (4, 6), (3, 1)]
	Corresponding inharmonicity coefficients: [  8.22087963e-04   5.38892819e-04   5.27078306e-05]

		Error! The true fingering was (3, 1) but we decided (5, 11)

	Fret n°2

		Estimated fundamental [Hz]: 220.0


C:\Users\rafilatty\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:675: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)


		Estimated inharmonicity constant: nan

	Potential fingerings candidates: [(5, 12), (4, 7), (3, 2)]
	Corresponding inharmonicity coefficients: [  9.22763486e-04   6.04887359e-04   5.91626002e-05]

		Error! The true fingering was (3, 2) but we decided (5, 12)

		Estimated fundamental [Hz]: 220.0


C:\Users\rafilatty\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:675: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)


		Estimated inharmonicity constant: nan

	Potential fingerings candidates: [(5, 12), (4, 7), (3, 2)]
	Corresponding inharmonicity coefficients: [  9.22763486e-04   6.04887359e-04   5.91626002e-05]

		Error! The true fingering was (3, 2) but we decided (5, 12)

		Estimated fundamental [Hz]: 220.0


C:\Users\rafilatty\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:675: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)


		Estimated inharmonicity constant: nan

	Potential fingerings candidates: [(5, 12), (4, 7), (3, 2)]
	Corresponding inharmonicity coefficients: [  9.22763486e-04   6.04887359e-04   5.91626002e-05]

		Error! The true fingering was (3, 2) but we decided (5, 12)

		Estimated fundamental [Hz]: 220.0


C:\Users\rafilatty\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:675: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)


		Estimated inharmonicity constant: nan

	Potential fingerings candidates: [(5, 12), (4, 7), (3, 2)]
	Corresponding inharmonicity coefficients: [  9.22763486e-04   6.04887359e-04   5.91626002e-05]

		Error! The true fingering was (3, 2) but we decided (5, 12)

		Estimated fundamental [Hz]: 220.0


C:\Users\rafilatty\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:675: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)


		Estimated inharmonicity constant: nan

	Potential fingerings candidates: [(5, 12), (4, 7), (3, 2)]
	Corresponding inharmonicity coefficients: [  9.22763486e-04   6.04887359e-04   5.91626002e-05]

		Error! The true fingering was (3, 2) but we decided (5, 12)

	Fret n°3

		Estimated fundamental [Hz]: 233.1


C:\Users\rafilatty\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:675: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)


		Estimated inharmonicity constant: nan

	Potential fingerings candidates: [(4, 8), (3, 3)]
	Corresponding inharmonicity coefficients: [  6.78963800e-04   6.64078415e-05]

		Error! The true fingering was (3, 3) but we decided (4, 8)

		Estimated fundamental [Hz]: 233.1


C:\Users\rafilatty\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:675: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)


		Estimated inharmonicity constant: nan

	Potential fingerings candidates: [(4, 8), (3, 3)]
	Corresponding inharmonicity coefficients: [  6.78963800e-04   6.64078415e-05]

		Error! The true fingering was (3, 3) but we decided (4, 8)

		Estimated fundamental [Hz]: 233.1
		Estimated inharmonicity constant: 8.106350605722318e-06

	Potential fingerings candidates: [(4, 8), (3, 3)]
	Corresponding inharmonicity coefficients: [  6.78963800e-04   6.64078415e-05]

		Fingering correctly estimated!

		Estimated fundamental [Hz]: 233.1
		Estimated inharmonicity constant: 2.3905707815727317e-06

	Potential fingerings candidates: [(4, 8), (3, 3)]
	Corresponding inharmonicity coefficients: [  6.78963800e-04   6.64078415e-05]

		Fingering correctly estimated!

		Estimated fundamental [Hz]: 233.1
		Estimated inharmonicity constant: 6.54872783007705e-06

	Potential fingerings candidates: [(4, 8), (3, 3)]
	Corresponding inharmonicity coefficients: [  6.78963800e-04   6.64078415e-05]

		Fingering correctl

C:\Users\rafilatty\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:675: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)


		Estimated inharmonicity constant: nan

	Potential fingerings candidates: [(4, 9), (3, 4), (2, 0)]
	Corresponding inharmonicity coefficients: [  7.62111880e-04   7.45403583e-05   4.89723809e-05]

		Error! The true fingering was (3, 4) but we decided (4, 9)

		Estimated fundamental [Hz]: 246.9
		Estimated inharmonicity constant: 0.00011896711961477186

	Potential fingerings candidates: [(4, 9), (3, 4), (2, 0)]
	Corresponding inharmonicity coefficients: [  7.62111880e-04   7.45403583e-05   4.89723809e-05]

		Fingering correctly estimated!

		Estimated fundamental [Hz]: 246.9
		Estimated inharmonicity constant: 3.888714434185499e-05

	Potential fingerings candidates: [(4, 9), (3, 4), (2, 0)]
	Corresponding inharmonicity coefficients: [  7.62111880e-04   7.45403583e-05   4.89723809e-05]

		Error! The true fingering was (3, 4) but we decided (2, 0)

		Estimated fundamental [Hz]: 246.9


C:\Users\rafilatty\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:675: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)


		Estimated inharmonicity constant: nan

	Potential fingerings candidates: [(4, 9), (3, 4), (2, 0)]
	Corresponding inharmonicity coefficients: [  7.62111880e-04   7.45403583e-05   4.89723809e-05]

		Error! The true fingering was (3, 4) but we decided (4, 9)

		Estimated fundamental [Hz]: 246.9
		Estimated inharmonicity constant: 6.557653990952307e-05

	Potential fingerings candidates: [(4, 9), (3, 4), (2, 0)]
	Corresponding inharmonicity coefficients: [  7.62111880e-04   7.45403583e-05   4.89723809e-05]

		Fingering correctly estimated!

	Fret n°5

		Estimated fundamental [Hz]: 261.6
		Estimated inharmonicity constant: 2.409630834856319e-05

	Potential fingerings candidates: [(4, 10), (3, 5), (2, 1)]
	Corresponding inharmonicity coefficients: [  8.55442540e-04   8.36688091e-05   5.49696954e-05]

		Error! The true fingering was (3, 5) but we decided (2, 1)

		Estimated fundamental [Hz]: 261.6
		Estimated inharmonicity constant: 1.0078449423869817e-05

	Potential fingerings candidates: [

C:\Users\rafilatty\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:675: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)


		Estimated inharmonicity constant: nan

	Potential fingerings candidates: [(4, 11), (3, 6), (2, 2)]
	Corresponding inharmonicity coefficients: [  9.60202771e-04   9.39151592e-05   6.17014602e-05]

		Error! The true fingering was (3, 6) but we decided (4, 11)

		Estimated fundamental [Hz]: 277.2
		Estimated inharmonicity constant: 9.50605479299523e-06

	Potential fingerings candidates: [(4, 11), (3, 6), (2, 2)]
	Corresponding inharmonicity coefficients: [  9.60202771e-04   9.39151592e-05   6.17014602e-05]

		Error! The true fingering was (3, 6) but we decided (2, 2)

		Estimated fundamental [Hz]: 277.2
		Estimated inharmonicity constant: 5.245621033623041e-06

	Potential fingerings candidates: [(4, 11), (3, 6), (2, 2)]
	Corresponding inharmonicity coefficients: [  9.60202771e-04   9.39151592e-05   6.17014602e-05]

		Error! The true fingering was (3, 6) but we decided (2, 2)

		Estimated fundamental [Hz]: 277.2


C:\Users\rafilatty\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:675: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)


		Estimated inharmonicity constant: nan

	Potential fingerings candidates: [(4, 11), (3, 6), (2, 2)]
	Corresponding inharmonicity coefficients: [  9.60202771e-04   9.39151592e-05   6.17014602e-05]

		Error! The true fingering was (3, 6) but we decided (4, 11)

		Estimated fundamental [Hz]: 277.2
		Estimated inharmonicity constant: 7.1520970388039284e-06

	Potential fingerings candidates: [(4, 11), (3, 6), (2, 2)]
	Corresponding inharmonicity coefficients: [  9.60202771e-04   9.39151592e-05   6.17014602e-05]

		Error! The true fingering was (3, 6) but we decided (2, 2)

	Fret n°7

		Estimated fundamental [Hz]: 293.7
		Estimated inharmonicity constant: 2.7755456422779516e-05

	Potential fingerings candidates: [(4, 12), (3, 7), (2, 3)]
	Corresponding inharmonicity coefficients: [  1.07779228e-03   1.05416310e-04   6.92576184e-05]

		Error! The true fingering was (3, 7) but we decided (2, 3)

		Estimated fundamental [Hz]: 293.7
		Estimated inharmonicity constant: 3.0232134899194843e-05

	P

In [24]:
print("\nTesting phase completed!")
print("\n\tTotal number of errors per string: {num_errors}".format(num_errors=num_errors_per_string))


Testing phase completed!

	Total number of errors per string: [ 14.  19.  57.  61.  29.  39.]


In [25]:
# OPTIONAL: store the results of the classification
#pickle.dump(num_errors_per_string, open('data_dumps/num_errors_per_string', 'wb'))

### VI.V Results <br />

<div style="text-align:justified;"> We now discuss the results of our simple classifier. </div>

In [1]:
num_errors_per_string_avg = pickle.load(open('data_dumps/num_errors_per_string_avg', 'rb'))
num_errors_per_string_poly = pickle.load(open('data_dumps/num_errors_per_string_poly', 'rb'))

# visualization: bar plot of the number of errors for each string
plt.figure(figsize=(14, 10))

ax1 = plt.subplot2grid((2, 2), (0, 0), rowspan=1, colspan=1)
ax1.bar(range(1, 7), num_errors_per_string_avg, color="blue", align='center')
plt.xlabel('String number')
plt.ylabel('Number of misclassifications')
plt.title('Number of misclassifications per string: Average method')

ax2 = plt.subplot2grid((2, 2), (0, 1), rowspan=1, colspan=1)
ax2.bar(range(1, 7), num_errors_per_string_poly, color="blue", align='center')
plt.xlabel('String number')
plt.ylabel('Number of misclassifications')
plt.title('Number of misclassifications per string: polynomial method')

plt.tight_layout()
plt.show()

NameError: name 'pickle' is not defined

<div style="text-align:justified;"> On the above plot, we display the results we achieved for both methods. We chose to consider the first 1.5 seconds of each recording, and the first 8 partials in frames of .75 seconds. The reason is that higher order partials are more volatile and harder to track. We used our modified Goertzel's algorithm to track the partials, as it was much faster and showed a more stable behavior. As we can see, the second method (polynomial fit) yields much better results. </div><br />

<div style="text-align:justified;"> Overall, the performance of our simple inharmonicity-based classifier is however mitigated. The number of misclassifications observed with the best method is still very high for the second and third strings (26 and 15 errors, respectively) and can be explained by the proximity of the two estimated constants $\hat{\gamma_2}$ and $\hat{\gamma_3}$. For these two, a slight imprecision in the inharmonicity estimation of a test recording will lead to a misclassification. </div>